## Loading libraries

In [1]:
import anndata as ad
import seaborn as sns
from matplotlib import pyplot as plt
import scvi
import numpy as np
import pandas as pd
from scipy import sparse
from scipy.stats import median_abs_deviation
import scanpy as sc
import os

Global seed set to 0


<h2>Loading data</h2>

<p style="color:orange; font-size:20px">Reading all the samples in the respective sample path.</p>

In [2]:
# Dir to the samples
sample_path = 'Samples'
# Folders between the sample name and the files (if doesn't exist, put '')
middle_path = '/filtered_feature_bc_matrix'

sample_names = []
for foldername in os.listdir(sample_path):
    if os.path.isdir(os.path.join(sample_path, foldername)):
        sample_names.append(foldername)
print(sample_names)
# For each sample, read the directory to a list of samples.
        
sample_list = []

for name in sample_names:
    # Getting anndata (transposed to obs X vars)
    path = f'samples/{name}{middle_path}/matrix.mtx.gz'
    sample = sc.read(path, cache=True).T
    
    # Getting obs
    path = f'samples/{name}{middle_path}/barcodes.tsv.gz'
    obs = pd.read_csv(path, sep='\t', header=None, index_col=0)
    obs.index.name = 'barcode'
    sample.obs = obs
    
    # Adding metadata
    sample.obs['Patient'] = name
    sample.obs['Condition'] = name[-1].upper()
    
    # Getting vars
    path = f"samples/{name}{middle_path}/features.tsv.gz"
    var = pd.read_table(path, sep='\t', header=None, index_col=1)
    var.index.name = 'genes'
    sample.var = var
    sample.var_names_make_unique(join="-")
    
    
    sample_list.append(sample)

['p018n', 'p018t', 'p019n', 'p019t', 'p023t', 'p024t', 'p027n', 'p027t', 'p028n', 'p029n', 'p030n', 'p030t', 'p031n', 'p031t', 'p032n', 'p032t', 'p033n', 'p033t', 'p034n', 'p034t']


In [3]:
adata = ad.concat(sample_list)
del sample_list
adata.obs_names_make_unique(join="-")
adata.obs['Author'] = 'Peng_2019'

C:\Users\athos\anaconda3\envs\scvi-env\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [6]:
adata.write_h5ad("adata_v1.0.h5ad")

## Removing doublets with SOLO (scVI)

<p style='color:orange; font-size:20px'>Removing cells with zero counts, to avoid errors in the model training.<\p>

In [7]:
adata.shape

(133736, 33538)

In [8]:
sc.pp.filter_cells(adata, min_counts=500)
sc.pp.filter_cells(adata, min_genes=200)
adata.shape

(131669, 33538)

In [9]:
adata.layers['counts'] = adata.X.copy()

<p style="color:orange; font-size:20px">Making the model.</p>

In [ ]:
scvi.model.SCVI.setup_anndata(adata, layer='counts', batch_key='Patient')
model = scvi.model.SCVI(adata)
model.train()

INFO     Using batches from adata.obs["Patient"]                                                                   
INFO     No label_key inputted, assuming all cells have same label                                                 
INFO     Using data from adata.layers["counts"]                                                                    
INFO     Successfully registered anndata object containing 131669 cells, 33538 vars, 20 batches, 1 labels, and 0   
         proteins. Also registered 0 extra categorical covariates and 0 extra continuous covariates.               
INFO     Please do not further modify adata until model is trained.                                                


GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Epoch 1/61:   0%|                                                                               | 0/61 [00:00<?, ?it/s]

<p style="color:orange; font-size:20px">Creating and training SOLO model.</p>

In [ ]:
solo = scvi.external.SOLO.from_scvi_model(model)
solo.train()

<p style="color:orange; font-size:20px">We then extract a predition dataframe. But as SOLO adds 2 characters in the barcode, we remove it for it to be the same format as Anndata.</p>

In [ ]:
df = solo.predict()
df['prediction'] = solo.predict(soft = False)
df

In [ ]:
df.groupby('prediction').count()
adata.obs['doublet_pred'] = df.prediction

<p style="color:orange; font-size:20px">Savind data for Seurat workflow.</p>

In [ ]:
adata.obs.to_csv("C:/Users/athos/UFCSPA/INICIAÇÃO CIENTÍFICA/Python Experiments/Lung Adenocarcinoma/metadata_workflow_peng.csv")

## Loading the raw data after SOLO

In [ ]:
metadata = pd.read_csv("C:/Users/athos/UFCSPA/INICIAÇÃO CIENTÍFICA/Python Experiments/Lung Adenocarcinoma/metadata_workflow_peng.csv", index_col = 0)
adata.obs = metadata
adata.obs

In [ ]:
adata = adata[adata.obs.doublet_pred == 'singlet'].copy()

## QC - Filtering low quality cells; run before SOLO, and after SOLO with the raw data (will be loaded again further in the downstream).

<p style="color:orange; font-size:20px">Calculating QC metrics.</p>

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
sc.pp.highly_variable_genes(adata, flavor='seurat_v3', n_top_genes=3000, layer='counts',subset=True, batch_key="Author")

In [ ]:
# mitochondrial genes
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes.
adata.var["hb"] = adata.var_names.str.contains(("^HB[^(P)]"))

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True
)
adata

In [ ]:
# plot1 = sns.displot(adata.obs["total_counts"], bins=100, kde=False)
# plot3 = sc.pl.scatter(adata, "total_counts", "n_genes_by_counts")

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_ribo', 'pct_counts_hb'],
         jitter=0.4, multi_panel=True)

<p style="color:orange; font-size:20px">Automatic threshold (outlier detection) with MAD.</p>

In [ ]:
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

In [ ]:
adata.obs["outlier"] = (
    is_outlier(adata, "log1p_total_counts", 5)
    | is_outlier(adata, "log1p_n_genes_by_counts", 5)
    | is_outlier(adata, "pct_counts_in_top_20_genes", 5)
)

adata.obs.outlier.value_counts()

In [ ]:
print(f"Total number of cells: {adata.n_obs}")
adata = adata[(~adata.obs.outlier)].copy()

print(f"Number of cells after filtering of low quality cells: {adata.n_obs}")

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20, )

## Integração com scVI

In [ ]:
scvi.model.SCVI.setup_anndata(adata, batch_key="Patient", layer="counts")

In [ ]:
arches_params = dict(
    use_layer_norm="both",
    use_batch_norm="none",
    encode_covariates=True,
    dropout_rate=0.2,
    n_layers=2,
)

vae = scvi.model.SCVI(adata, **arches_params)
vae.train()

In [ ]:
adata.obsm["X_scVI"] = vae.get_latent_representation()
adata.layers['scvi_normalized'] = vae.get_normalized_expression(library_size = 1e4)
sc.pp.neighbors(adata, use_rep="X_scVI")
sc.tl.umap(adata)

## Umap Visualization

<p style="color:orange; font-size:20px">Visualizing Batch Correction.</p>

In [ ]:
sc.pl.umap(adata, color = ['Condition', 'Patient'], frameon=False, ncols = 2)

<p style="color:orange; font-size:20px">Visualizing canonic genes. → Section in umap-analysis-1.ipynb</p>

<p style="color:orange; font-size:20px">Changing resolution.</p>

In [ ]:
for resolution in [0.2, 0.5, 1.0, 1.5]:
    sc.tl.leiden(adata, resolution=resolution, key_added=f"leiden_r{resolution}", random_state=0)

In [ ]:
sc.pl.umap(adata, color=['leiden_r0.2','leiden_r0.5', 'leiden_r1.0', 'leiden_r1.5'], frameon=False, ncols=1)

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden_r0.2', method='wilcoxon')
markers = sc.get.rank_genes_groups_df(adata, None)
markers = markers[(markers.pvals_adj < 0.05) & (markers.logfoldchanges > .5)]

In [ ]:
markers

<p style="color:orange; font-size:20px">Defining and analysing cell markers.</p>

In [ ]:
print(adata.var_names)

In [ ]:
cell_markers = {
    'Immune': ['S100A8', 'LYZ', 'FABP4', 'CD14'],
    'Tumor' : ['EGFR', 'TFF3', 'CDKN2A', 'SFTPA2'],
    'Stroma': ['PROX1', 'CCL21', 'PDGFRA', 'CLDN5', 'VWF', 'PLVAP'],
    'Epithelial': ['CALCA', 'MUC5B', 'KRT5'],
}

# Could not find keys '['AIF1' (my), 'CD3E' (tnk), 'CD68' (my), 'CD79B' (b), 'CD8A' (tnk), 'FOXP3', (tnk)]'

In [ ]:
sc.pl.dotplot(adata, cell_markers, 'leiden_r0.2', dendrogram=False)
sc.pl.dotplot(adata, cell_markers, 'leiden_r0.5', dendrogram=False)
sc.pl.dotplot(adata, cell_markers, 'leiden_r1.0', dendrogram=False)
sc.pl.dotplot(adata, cell_markers, 'leiden_r1.5', dendrogram=False)